In [ ]:
! pip install --upgrade google-cloud-bigquery-storage

In [2]:
import google, pathlib, shutil, urllib, zipfile as zf, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from shapely.ops import orient
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
root_path = pathlib.Path(root_path)
raw_data_path = root_path / 'raw_data'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

def read_file(zipfile, f, **kwargs):
    file = zipfile.extract(f)
    df = pd.read_csv(file, dtype=str, **kwargs)
    df.columns = [str(x).lower() for x in df.columns]
    return df

def get_states():
    qry = f"""
    select
        state_fips_code as fips
        , state_postal_abbreviation as abbr
        , state_name as name
    from
        bigquery-public-data.census_utility.fips_codes_states
    where
        state_fips_code <= '56'
    """
    return bqclient.query(qry).result().to_dataframe()

def get_data(name, url, state, yr=2020, overwrite=False):
    print(f"\nGet {state['name']} {yr} {name}", end='...' )
    path = f"{name}/{yr}/{state['abbr']}"
    zipfile = raw_data_path / f"{path}/{path.replace('/', '_')}.zip"
    path = zipfile.parent
    
    if overwrite:
        shutil.rmtree(path, ignore_errors=True)
    path.mkdir(parents=True, exist_ok=True)
    %cd {path}
    try:
        zf.ZipFile(zipfile)
        print('already have zip')
    except:
        try:
            print(f'fetching zip from {url}', end = '...')
            zf.ZipFile(urllib.request.urlretrieve(url, zipfile)[0])
            print('success')
        except urllib.error.HTTPError:
            print('\n\nFAILED - BAD URL\n\n')
            zipfile = None
    parquetfile = zipfile.with_suffix('.parquet')
    try:
        df = gpd.read_parquet(parquetfile)
    except:
        try:
            df = pd.read_parquet(parquetfile)
        except:
            df = None
    return zipfile, df

def write_data(df, file):
    df.to_parquet(file.with_suffix('.parquet'))
    table_id = f"{proj_id}.redistricting.{file.stem}"
    bqclient.delete_table(table_id, not_found_ok=True)
    job = bqclient.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for the job to complete.

states = get_states()
states = states.iloc[[39,43]]
overwrite = False
# overwrite = True


In [14]:
def get_crosswalks(state, overwrite=False):
    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{state['fips']}.zip"
    yr = 2020
    zipfile, df = get_data(name, url, state, yr, overwrite)
    
    if df is None and zipfile is not None:
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            if file[-3:] == 'txt':
                df = read_file(Z, file, sep='|')
                for yr in [2010, 2020]:
                    df[f'geoid_{yr}'] = df[f'state_{yr}'].str.rjust(2,'0') + df[f'county_{yr}'].str.rjust(3,'0') + df[f'tract_{yr}'].str.rjust(6,'0') + df[f'blk_{yr}'].str.rjust(4,'0')
                df = df[['geoid_2010', 'geoid_2020']]
                display(df.head(2))
        write_data(df, zipfile)
    return df

def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            col = file.lower().split('_')[-1][:-4]
            if file[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(Z, file, sep=sep)
                if col == 'vtd':
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        write_data(df, zipfile)
    return df

def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            w = file.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(Z, file)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, zipfile)
    return df

def get_census(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            #     source_format=bigquery.SourceFormat.CSV,
        )
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            table_id = f"{proj_id}.redistricting.{zipfile.stem}_"
            print(file)
            if file[2:5] == 'geo':
                Z.extract(file)
                temp_table_id = table_id + 'temp'
                with open(file, mode="rb") as f:
                    load_job = bqclient.load_table_from_file(f, temp_table_id, job_config=job_config)
                    load_job.result()

                widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
                names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                sel = [f'trim(substring(string_field_0, {s}, {w})) as {n}' for s, w, n in zip(starts, widths, names)]
                qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id
                
                bqclient.delete_table(table_id+'geo', not_found_ok=True)
                query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id+'geo', autodetect=True))
                query_job.result()
                bqclient.delete_table(temp_table_id)
            else:
                names = [['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],
                         ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18',
                          'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']]
                i = file[6]
                if i in ['1', '2']:
                    Z.extract(file)
                    cmd = 'sed -i "1s/^/' + ','.join(names[int(i)-1]) + '\\n/" ' + file
                    ! {cmd}
                    with open(file, mode="rb") as f:
                        load_job = bqclient.load_table_from_file(f, table_id+i, job_config=job_config)
                        load_job.result()
        ja = '\n, A.'
        jb = '\n, B.'
        qry = f"""
        select
            concat(right(concat("00",C.state), 2), right(concat("000",C.county), 3), right(concat("000000",C.tract), 6), right(concat("0000",C.block), 4)) as geoid
            , C.*
            , A.{ja.join(names[0][5:])}
            , B.{jb.join(names[1][5:])}
        from
            {table_id}1 as A
        inner join
            {table_id}2 as B
        on
            A.fileid = B.fileid
            and A.stusab = B.stusab
            and A.logrecno = B.logrecno
        inner join
            {table_id}geo as C
        on
            A.fileid = trim(C.fileid)
            and A.stusab = trim(C.stusab)
            and A.logrecno = cast(C.logrecno as int)
        where
            C.block != ""
        order by
            A.logrecno
        """
        print(qry)
        bqclient.delete_table(table_id+'comb', not_found_ok=True)
        query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id+'comb'))
        query_job.result()
                         
        df = pd.DataFrame()
        df['A'] = [1]
        df.to_parquet(zipfile.with_suffix('.parquet'))
    return df

elections = get_elections_TX(overwrite)
for i, state in states.iterrows():
    crosswalks = get_crosswalks(state, overwrite)
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)
        if yr == 2010:
            get_census(state, yr, overwrite)


Get Rhode Island 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/RI
already have zip

Get Rhode Island 2020 assignments.../home/jupyter/raw_data/assignments/2020/RI
already have zip

Get Rhode Island 2010 assignments.../home/jupyter/raw_data/assignments/2010/RI
already have zip

Get Rhode Island 2010 census.../home/jupyter/raw_data/census/2010/RI
already have zip
rigeo2010.pl
ri000012010.pl
ri000022010.pl
ri2010.pl.prd.packinglist.txt

        select
            concat(right(concat("00",C.state), 2), right(concat("000",C.county), 3), right(concat("000000",C.tract), 6), right(concat("0000",C.block), 4)) as geoid
            , C.*
            , A.total
    , A.population_of_one_race
    , A.white_alone
    , A.black_or_african_american_alone
    , A.american_indian_and_alaska_native_alone
    , A.asian_alone
    , A.native_hawaiian_and_other_pacific_islander_alone
    , A.some_other_race_alone
    , A.population_of_two_or_more_races
    , A.population_of_two_races
    , A.white

In [3]:
# def get_census_2010():
def get_census(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            #     source_format=bigquery.SourceFormat.CSV,
        )
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            table_id = f"{proj_id}.redistricting.{zipfile.stem}_"
            print(file)
            if file[2:5] == 'geo':
                Z.extract(file)
                temp_table_id = table_id + 'temp'
                with open(file, mode="rb") as f:
                    load_job = bqclient.load_table_from_file(f, temp_table_id, job_config=job_config)
                    load_job.result()

                widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
                names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                sel = [f'substring(string_field_0, {s}, {w}) as {n}' for s, w, n in zip(starts, widths, names)]
                qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id
#                 print(qry)
                
                table_id += 'geo'
                query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id))
                query_job.result()
#                 bqclient.delete_table(temp_table_id)
            else:
                names = [['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],
                         ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']]
                i = file[6]
                if i in ['1', '2']:
                     Z.extract(file)
                     cmd = 'sed -i "1s/^/' + ','.join(names[int(i)-1]) + '\\n/" ' + file
                     ! {cmd}
                     table_id += i
                     with open(file, mode="rb") as f:
                         load_job = bqclient.load_table_from_file(f, table_id, job_config=job_config)
                         load_job.result()
for i, state in states.iterrows():
    for yr in [2010]:#, 2020]:
        census = get_census(state, yr, overwrite)

Get Rhode Island 2010 census.../home/jupyter/raw_data/census/2010/RI

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Rhode_Island/ri2010.pl.zip...success
rigeo2010.pl
ri000012010.pl
ri000022010.pl
ri2010.pl.prd.packinglist.txt
Get Texas 2010 census.../home/jupyter/raw_data/census/2010/TX

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Texas/tx2010.pl.zip...success
txgeo2010.pl
tx000012010.pl
tx000022010.pl
tx2010.pl.prd.packinglist.txt


In [38]:
temp_table_id = f"{proj_id}.cook_test.temp"

widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
sel = [f'substring(string_field_0, {s}, {w}) as {n}' for s, w, n in zip(starts, widths, names)]
qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id

table_id = f"{proj_id}.cook_test.{zipfile.stem}{geo}"
job_config = bigquery.QueryJobConfig(destination=table_id)
query_job = client.query(qry, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print(qry)

select
	substring(string_field_0, 1, 6) as fileid,
	substring(string_field_0, 7, 2) as stusab,
	substring(string_field_0, 9, 3) as sumlev,
	substring(string_field_0, 12, 2) as geocomp,
	substring(string_field_0, 14, 3) as chariter,
	substring(string_field_0, 17, 2) as cifsn,
	substring(string_field_0, 19, 7) as logrecno,
	substring(string_field_0, 26, 1) as region,
	substring(string_field_0, 27, 1) as division,
	substring(string_field_0, 28, 2) as state,
	substring(string_field_0, 30, 3) as county,
	substring(string_field_0, 33, 2) as countycc,
	substring(string_field_0, 35, 2) as countysc,
	substring(string_field_0, 37, 5) as cousub,
	substring(string_field_0, 42, 2) as cousubcc,
	substring(string_field_0, 44, 2) as cousubsc,
	substring(string_field_0, 46, 5) as place,
	substring(string_field_0, 51, 2) as placecc,
	substring(string_field_0, 53, 2) as placesc,
	substring(string_field_0, 55, 6) as tract,
	substring(string_field_0, 61, 1) as blkgrp,
	substring(string_field_0, 62, 4) as b

In [22]:
s = [1,2,3]
np.insert(np.cumsum(s)[:-1], 0 ,0)+1
np.cumsum(s)
# np.insert?

array([1, 3, 6])

In [15]:
L = ['1', '2']
L.index

Signature: L.index(value, start=0, stop=9223372036854775807, /)
Docstring:
Return first index of value.

Raises ValueError if the value is not present.
Type:      builtin_function_or_method


In [12]:
s = 'ri000012010.pl'
s[6] == 1

False

In [5]:
path = raw_data_path / 'census/2010/RI'
file = path /'ri000012010.pl'

table_id = f"{proj_id}.cook_test.{file.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

%cd {file.parent}
names1 = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl']
names2 = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']
cmd = 'sed -i "1s/^/' + ','.join(names) + '\\n/" ' + file.name
! {cmd}

/home/jupyter/raw_data/census/2010/RI


In [ ]:
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_HL', 'hispanic_or_latino_HL', 'not_hispanic_or_latino_HL', 'population_of_one_race_HL', 'white_alone_HL', 'black_or_african_american_alone_HL', 'american_indian_and_alaska_native_alone_HL', 'asian_alone_HL', 'native_hawaiian_and_other_pacific_islander_alone_HL', 'some_other_race_alone_HL', 'population_of_two_or_more_races_HL', 'population_of_two_races_HL', 'white_black_or_african_american_HL', 'white_american_indian_and_alaska_native_HL', 'white_asian_HL', 'white_native_hawaiian_and_other_pacific_islander_HL', 'white_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_HL', 'black_or_african_american_asian_HL', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_some_other_race_HL', 'american_indian_and_alaska_native_asian_HL', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'american_indian_and_alaska_native_some_other_race_HL', 'asian_native_hawaiian_and_other_pacific_islander_HL', 'asian_some_other_race_HL', 'native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_three_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_HL', 'white_black_or_african_american_asian_HL', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_HL', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'white_american_indian_and_alaska_native_some_other_race_HL', 'white_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_asian_some_other_race_HL', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_HL', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_HL', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_asian_some_other_race_HL', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'american_indian_and_alaska_native_asian_some_other_race_HL', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_four_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_HL', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_HL', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_asian_some_other_race_HL', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_american_indian_and_alaska_native_asian_some_other_race_HL', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_five_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_HL', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_six_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL']
names = [c.replace(' ', '_').replace(';', '_').lower() for c in names]
print(names)

In [9]:
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
#     source_format=bigquery.SourceFormat.CSV,
)

with open(file, mode="rb") as f:
    load_job = bqclient.load_table_from_file(f, table_id, job_config=job_config)
    load_job.result()

In [151]:
# df = pd.read_csv(file, nrows=100)
# # df.columns = L
# # df.isnull().any().any()
# # df.shape
# job = bqclient.load_table_from_dataframe(df, table_id)
# job.result()  # Wait for the job to complete.

# bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)


with open(file, mode="rb") as f:
    load_job = bqclient.load_table_from_file(
        f, table_id,
        job_config=job_config
    )
    load_job.result() 
#     df = pd.read_csv(f, nrows=2, header=None)

In [152]:
df = pd.read_csv(file, nrows=100)
df

,fileid,stusab,chariter,cifsn,logrecno,total,population_of_one_race,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,...,american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL,population_of_five_races_HL,white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL,white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_HL,white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL,white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL,white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL,black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL,population_of_six_races_HL,white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL
0,PLST,RI,0,1,1,1052567,1017780,856869,60189,6058,...,1,22,3,18,0,0,1,0,0,0
1,PLST,RI,0,1,2,519021,500287,424536,31676,2357,...,1,22,3,18,0,0,1,0,0,0
2,PLST,RI,0,1,3,49875,49134,47752,398,80,...,0,0,0,0,0,0,0,0,0,0
3,PLST,RI,0,1,4,4654,4562,4384,26,7,...,0,0,0,0,0,0,0,0,0,0
4,PLST,RI,0,1,5,3188,3142,2955,30,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PLST,RI,0,1,96,5935,5804,5310,227,15,...,0,1,1,0,0,0,0,0,0,0
96,PLST,RI,0,1,97,4325,4163,3648,229,22,...,0,0,0,0,0,0,0,0,0,0
97,PLST,RI,0,1,98,2964,2915,2594,136,8,...,0,0,0,0,0,0,0,0,0,0
98,PLST,RI,0,1,99,4218,4099,3788,132,21,...,0,0,0,0,0,0,0,0,0,0


In [149]:
df.dtypes

fileid                                                                                                                                   object
stusab                                                                                                                                   object
chariter                                                                                                                                  int64
cifsn                                                                                                                                     int64
logrecno                                                                                                                                  int64
                                                                                                                                          ...  
white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL                                     

In [ ]:
bigquery.LoadJobConfig??

In [94]:
t = bqclient.get_table(table_id)
df = bqclient.list_rows(t).to_dataframe()

In [96]:
df.head(10)
# df['int64_field_5'].unique()

,string_field_0,string_field_1,int64_field_2,int64_field_3,int64_field_4,int64_field_5,int64_field_6,int64_field_7,int64_field_8,int64_field_9,...,int64_field_139,int64_field_140,int64_field_141,int64_field_142,int64_field_143,int64_field_144,int64_field_145,int64_field_146,int64_field_147,int64_field_148
0,PLST,RI,0,1,41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PLST,RI,0,1,42,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PLST,RI,0,1,191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PLST,RI,0,1,329,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PLST,RI,0,1,330,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,PLST,RI,0,1,362,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,PLST,RI,0,1,388,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,PLST,RI,0,1,421,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,PLST,RI,0,1,424,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,PLST,RI,0,1,440,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
cmd = 'sed -i "1s/^/' + ','.join(names) + '\n/"' + 
cmd


In [52]:
df = pd.read_csv(file, nrows=10)
df

,fileid,stusab,chariter,cifsn,logrecno,total,population of one race,white alone,black or african american alone,american indian and alaska native alone,asian alone,native hawaiian and other pacific islander alone,some other race alone,population of two or more races,population of two races,white; black or african american,white; american indian and alaska native,white; asian,white; native hawaiian and other pacific islander,white; some other race,black or african american; american indian and alaska native,black or african american; asian,black or african american; native hawaiian and other pacific islander,black or african american; some other race,american indian and alaska native; asian,american indian and alaska native; native hawaiian and other pacific islander,american indian and alaska native; some other race,asian; native hawaiian and other pacific islander,asian; some other race,native hawaiian and other pacific islander; some other race,population of three races,white; black or african american; american indian and alaska native,white; black or african american; asian,white; black or african american; native hawaiian and other pacific islander,white; black or african american; some other race,white; american indian and alaska native; asian,white; american indian and alaska native; native hawaiian and other pacific islander,white; american indian and alaska native; some other race,white; asian; native hawaiian and other pacific islander,white; asian; some other race,white; native hawaiian and other pacific islander; some other race,black or african american; american indian and alaska native; asian,black or african american; american indian and alaska native; native hawaiian and other pacific islander,black or african american; american indian and alaska native; some other race,black or african american; asian; native hawaiian and other pacific islander,black or african american; asian; some other race,black or african american; native hawaiian and other pacific islander; some other race,american indian and alaska native; asian; native hawaiian and other pacific islander,american indian and alaska native; asian; some other race,american indian and alaska native; native hawaiian and other pacific islander; some other race,asian; native hawaiian and other pacific islander; some other race,population of four races,white; black or african american; american indian and alaska native; asian,white; black or african american; american indian and alaska native; native hawaiian and other pacific islander,white; black or african american; american indian and alaska native; some other race,white; black or african american; asian; native hawaiian and other pacific islander,white; black or african american; asian; some other race,white; black or african american; native hawaiian and other pacific islander; some other race,white; american indian and alaska native; asian; native hawaiian and other pacific islander,white; american indian and alaska native; asian; some other race,white; american indian and alaska native; native hawaiian and other pacific islander; some other race,white; asian; native hawaiian and other pacific islander; some other race,black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander,black or african american; american indian and alaska native; asian; some other race,black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race,black or african american; asian; native hawaiian and other pacific islander; some other race,american indian and alaska native; asian; native hawaiian and other pacific islander; some other race,population of five races,white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander,white; black or african american; american indian and alaska native; asian; some other race,white; black or african american; american indi

In [56]:
file

'/home/jupyter/raw_data/census/2010/TX/tx000012010.pl'

In [57]:
%ls
table_id = f"{proj_id}.cook_test.{parquetfile.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 

census_2010_TX.zip  tx000022010.pl                 txgeo2010.pl
tx000012010.pl      tx2010.pl.prd.packinglist.txt


In [ ]:
# zf.extract_all(zipfile)

# zipfile.extractall()
for f in zipfile.namelist():
    if f[-2:] == 'pl':
#         file = zipfile.extract(f)
        if f[2:5] == 'geo':
            pass
#             widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
#             names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
#             df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
#             df = df.query('block != ""')
        elif f[6] == '1':
            names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
            cmd = 'sed -i "1s/^/' + ','.join(names) + '\\n/" ' + file
            print(cmd)
            ! {cmd}
#             names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
#             df = read_file(zipfile, f          , header=None, nrows=nrows)
        elif f[6] == '2':
            pass
#             names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
#             df = read_file(zipfile, f          , header=None, nrows=nrows)



#     widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
#     names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
#     df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
#     df = df.query('block != ""')
# elif f[6] == '1':
#     names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
#     df = read_file(zipfile, f          , header=None, nrows=nrows)
# elif f[6] == '2':
#     names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
#     df = read_file(zipfile, f          , header=None, nrows=nrows)



In [20]:
def get_crosswalks(state, overwrite=False):
    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{state['fips']}.zip"
    yr = 2020
    zipfile, df = get_data(name, url, state, yr, overwrite)
    if df is None and zipfile is not None:
        for f in zipfile.namelist():
            if f[-3:] == 'txt':
                df = read_file(zipfile, f, sep='|')
                for yr in [2010, 2020]:
                    df[f'geoid_{yr}'] = df[f'state_{yr}'].str.rjust(2,'0') + df[f'county_{yr}'].str.rjust(3,'0') + df[f'tract_{yr}'].str.rjust(6,'0') + df[f'blk_{yr}'].str.rjust(4,'0')
                df = df[['geoid_2010', 'geoid_2020']]
                display(df.head(2))
        write_data(df, zipfile)
    return df

def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            col = f.lower().split('_')[-1][:-4]
            print(col)
            if f[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(zipfile, f, sep=sep)
                
                if col == 'vtd':
                    display(df.head(2))
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        display(df.head(5))
        write_data(df, zipfile)
    return df

def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            w = f.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(zipfile, f)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, zipfile)
    return df

elections = get_elections_TX(overwrite)
for i, state in states.iterrows():
    crosswalks = get_crosswalks(state, overwrite)
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)

Get Rhode Island 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/RI
already have zip


,geoid_2010,geoid_2020
0,090117052003000,440090510001039
1,250056303002046,440070114041025


Get Texas 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/TX
already have zip


,geoid_2010,geoid_2020
0,050910209001225,480370112012022
1,050910210007000,480679501013066


In [22]:
    job = bqclient.load_table_from_dataframe(df, table_id)

'crosswalks_2020_TX'

In [21]:
%ls
table_id = f"{proj_id}.cook_test.{parquetfile.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 


crosswalks_2020_TX.parquet  crosswalks_2020_TX.zip  tab2010_tab2020_st48_tx.txt


In [16]:
url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/2020-general-vtd-election-data.zip'
zipfile = root_path / 'temp/data.zip'
%cd {zipfile.parent}
# z = urllib.request.urlretrieve(url, zipfile)[0]
zipfile = zf.ZipFile(z)
for f in zipfile.namelist():
    w = f.split('_')
    if w.pop(-1) == 'Returns.csv':
        print(f)
    

/home/jupyter/temp
2012_Democratic_Primary_Election_Returns.csv
2012_Democratic_Runoff_Election_Returns.csv
2012_General_Election_Returns.csv
2012_Republican_Primary_Election_Returns.csv
2012_Republican_Runoff_Election_Returns.csv
2013_Constitutional_Amend_Election_Returns.csv
2014_Constitutional_Amend_Election_Returns.csv
2014_Democratic_Primary_Election_Returns.csv
2014_Democratic_Runoff_Election_Returns.csv
2014_General_Election_Returns.csv
2014_Republican_Primary_Election_Returns.csv
2014_Republican_Runoff_Election_Returns.csv
2014_Senate_Dist_18_Election_Returns.csv
2015_House_Dist_123_(2nd)_Election_Returns.csv
2015_House_Dist_123_Election_Returns.csv
2015_House_Dist_124_(2nd)_Election_Returns.csv
2015_House_Dist_124_Election_Returns.csv
2015_House_Dist_13_(2nd)_Election_Returns.csv
2015_House_Dist_13_Election_Returns.csv
2015_House_Dist_17_(2nd)_Election_Returns.csv
2015_House_Dist_17_Election_Returns.csv
2015_House_District_118_Election_Returns.csv
2015_Senate_Dist_26_(2nd)_Ele

In [6]:
for 

<zipfile.ZipFile filename='/home/jupyter/temp/data.zip' mode='r'>

In [74]:
abbr = 'TX'
# file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
# file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}000022010.pl'
# file = raw_data_path / f'elections/2020/{abbr}/elections_2020_TX.parquet'
file = raw_data_path / f'elections/2020/{abbr}/2020_General_Election_Returns.csv'

file.is_file()

df = pd.read_csv(file, nrows=10)
df
display(df)

table_id = f"{proj_id}.redistricting.{file.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 

# bqclient.load_table_from_dataframe(df, table_id)


# with open(file, mode='rb') as f:
#     bqclient.load_table_from_file(f, table_id)
# # bqclient.load_table_from_file?
# # bqclient.load_table_from_uri?

,County,FIPS,VTD,cntyvtd,Office,Name,Party,Incumbent,Votes
0,Anderson,1,1,10001,President,Biden,D,N,357
1,Anderson,1,2,10002,President,Biden,D,N,205
2,Anderson,1,3,10003,President,Biden,D,N,118
3,Anderson,1,4,10004,President,Biden,D,N,117
4,Anderson,1,5,10005,President,Biden,D,N,12
5,Anderson,1,6,10006,President,Biden,D,N,389
6,Anderson,1,7,10007,President,Biden,D,N,462
7,Anderson,1,8,10008,President,Biden,D,N,92
8,Anderson,1,9,10009,President,Biden,D,N,160
9,Anderson,1,10,10010,President,Biden,D,N,42


BadRequest: 400 Error while reading data, error message: CSV table references column position 8, but line starting at position:24468116 contains only 1 columns.

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def get_census(state, yr, overwrite=False, nrows=None):
    yrs = [2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    name = f'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        if yr == 2010:
            idx = ['fileid', 'stusab', 'chariter', 'logrecno']
            for f in zipfile.namelist():
                if f[-2:] == 'pl':
                    if f[2:5] == 'geo':
                        widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                        names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                        df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
                        df = df.query('block != ""')
                    elif f[6] == '1':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    elif f[6] == '2':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    df.columns = names
                    L.append(df.drop(columns='cifsn').set_index(idx))
            df = pd.concat(L, axis=1, how='inner').reset_index()
            df.columns = [c.replace(' ', '_').replace(';', '_') for c in df.columns]
        display(df.head(3))
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2010]:#, 2020]:
        census = get_census(state, yr, overwrite, 100)

Get Rhode Island 2010 census.../home/jupyter/raw_data/census/2010/RI

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Rhode_Island/ri2010.pl.zip...success


,fileid,stusab,chariter,logrecno,sumlev,geocomp,region,division,state,county,countycc,countysc,cousub,cousubcc,cousubsc,place,placecc,placesc,tract,blkgrp,block,iuc,concit,concitcc,concitsc,aianhh,aianhhfp,aianhhcc,aihhtli,aitsce,aits,aitscc,ttract,tblkgrp,anrc,anrccc,cbsa,cbsasc,metdiv,csa,necta,nectasc,nectadiv,cnecta,cbsapci,nectapci,ua,uasc,uatype,ur,cd,sldu,sldl,vtd,vtdi,reserve2,zcta5,submcd,submcdcc,sdelm,sdsec,sduni,arealand,areawatr,name,funcstat,gcuni,pop100,hu100,intptlat,intptlon,lsadc,partflag,reserve3,uga,statens,countyns,cousubns,placens,concitns,aianhhns,aitsns,anrcns,submcdns,cd113,cd114,cd115,sldu2,sldu3,sldu4,sldl2,sldl3,sldl4,aianhhsc,csasc,cnectasc,memi,nmemi,puma,reserved,total,population_of_one_race,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,population_of_two_or_more_races,population_of_two_races,white__black_or_african_american,white__american_indian_and_alaska_native,white__asian,white__native_hawaiian_and_other_pacific_islander,white__some_other_race,black_or_african_american__american_indian_and_alaska_native,black_or_african_american__asian,black_or_african_american__native_hawaiian_and_other_pacific_islander,black_or_african_american__some_other_race,american_indian_and_alaska_native__asian,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__some_other_race,asian__native_hawaiian_and_other_pacific_islander,asian__some_other_race,native_hawaiian_and_other_pacific_islander__some_other_race,population_of_three_races,white__black_or_african_american__american_indian_and_alaska_native,white__black_or_african_american__asian,white__black_or_african_american__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__some_other_race,white__american_indian_and_alaska_native__asian,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander,white__asian__some_other_race,white__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian,black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,black_or_african_american__american_indian_and_alaska_native__some_other_race,black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,black_or_african_american__asian__some_other_race,black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__asian__some_other_race,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,asian__native_hawaiian_and_other_pacific_islander__some_other_race,population_of_four_races,white__black_or_african_american__american_indian_and_alaska_native__asian,white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__american_indian_and_alaska_native__some_other_race,white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__asian__some_other_race,white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__asian__some_other_race,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,black_or_african_ameri

Get Texas 2010 census.../home/jupyter/raw_data/census/2010/TX

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Texas/tx2010.pl.zip...success


,fileid,stusab,chariter,logrecno,sumlev,geocomp,region,division,state,county,countycc,countysc,cousub,cousubcc,cousubsc,place,placecc,placesc,tract,blkgrp,block,iuc,concit,concitcc,concitsc,aianhh,aianhhfp,aianhhcc,aihhtli,aitsce,aits,aitscc,ttract,tblkgrp,anrc,anrccc,cbsa,cbsasc,metdiv,csa,necta,nectasc,nectadiv,cnecta,cbsapci,nectapci,ua,uasc,uatype,ur,cd,sldu,sldl,vtd,vtdi,reserve2,zcta5,submcd,submcdcc,sdelm,sdsec,sduni,arealand,areawatr,name,funcstat,gcuni,pop100,hu100,intptlat,intptlon,lsadc,partflag,reserve3,uga,statens,countyns,cousubns,placens,concitns,aianhhns,aitsns,anrcns,submcdns,cd113,cd114,cd115,sldu2,sldu3,sldu4,sldl2,sldl3,sldl4,aianhhsc,csasc,cnectasc,memi,nmemi,puma,reserved,total,population_of_one_race,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,population_of_two_or_more_races,population_of_two_races,white__black_or_african_american,white__american_indian_and_alaska_native,white__asian,white__native_hawaiian_and_other_pacific_islander,white__some_other_race,black_or_african_american__american_indian_and_alaska_native,black_or_african_american__asian,black_or_african_american__native_hawaiian_and_other_pacific_islander,black_or_african_american__some_other_race,american_indian_and_alaska_native__asian,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__some_other_race,asian__native_hawaiian_and_other_pacific_islander,asian__some_other_race,native_hawaiian_and_other_pacific_islander__some_other_race,population_of_three_races,white__black_or_african_american__american_indian_and_alaska_native,white__black_or_african_american__asian,white__black_or_african_american__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__some_other_race,white__american_indian_and_alaska_native__asian,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander,white__asian__some_other_race,white__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian,black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,black_or_african_american__american_indian_and_alaska_native__some_other_race,black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,black_or_african_american__asian__some_other_race,black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__asian__some_other_race,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,asian__native_hawaiian_and_other_pacific_islander__some_other_race,population_of_four_races,white__black_or_african_american__american_indian_and_alaska_native__asian,white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__american_indian_and_alaska_native__some_other_race,white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__asian__some_other_race,white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__asian__some_other_race,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,black_or_african_ameri

In [77]:
df#.isnull().any()

,County,FIPS,VTD,cntyvtd,Office,Name,Party,Incumbent,Votes
0,Anderson,1,1,10001,President,Biden,D,N,357
1,Anderson,1,2,10002,President,Biden,D,N,205
2,Anderson,1,3,10003,President,Biden,D,N,118
3,Anderson,1,4,10004,President,Biden,D,N,117
4,Anderson,1,5,10005,President,Biden,D,N,12
5,Anderson,1,6,10006,President,Biden,D,N,389
6,Anderson,1,7,10007,President,Biden,D,N,462
7,Anderson,1,8,10008,President,Biden,D,N,92
8,Anderson,1,9,10009,President,Biden,D,N,160
9,Anderson,1,10,10010,President,Biden,D,N,42


In [63]:
# pd.read_gbq()
t = bqclient.get_table(table_id)

In [52]:
bqclient.get_table(table_id)

Table(TableReference(DatasetReference('cmat-315920', 'redistricting'), '2020_General_Election_Returns'))

In [65]:
df = bqclient.list_rows(t).to_dataframe()
# bqclient.

In [67]:
bqclient.create_table?

Signature:
bqclient.create_table(
    table,
    exists_ok=False,
    retry=<google.api_core.retry.Retry object at 0x7f0887682150>,
    timeout=None,
)
Docstring:
API call:  create a table via a PUT request

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/tables/insert

Args:
    table (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
        A :class:`~google.cloud.bigquery.table.Table` to create.
        If ``table`` is a reference, an empty table is created
        with the specified ID. The dataset that the table belongs to
        must already exist.
    exists_ok (Optional[bool]):
        Defaults to ``False``. If ``True``, ignore "already exists"
        errors when creating the table.
    retry (Optional[google.api_core.retry.Retry]):
        How to retry the RPC.
    timeout (Optional[float]):
        The number of seconds to wait for the underlying HTTP trans

In [50]:
bqclient.load_table_from_file?

Signature:
bqclient.load_table_from_file(
    file_obj,
    destination,
    rewind=False,
    size=None,
    num_retries=6,
    job_id=None,
    job_id_prefix=None,
    location=None,
    project=None,
    job_config=None,
    timeout=None,
)
Docstring:
Upload the contents of this table from a file-like object.

Similar to :meth:`load_table_from_uri`, this method creates, starts and
returns a :class:`~google.cloud.bigquery.job.LoadJob`.

Args:
    file_obj (file): A file handle opened in binary mode for reading.
    destination (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
        Table into which data is to be loaded. If a string is passed
        in, this method attempts to create a table reference from a
        string using
        :func:`google.cloud.bigquery.table.TableReference.from_string`.

Keyword Arguments:
    rewind (Optional[bool]):
        If True, seek to the b

In [31]:
# bqclient.load_table_from_file?
bqclient.create_table?

Signature:
bqclient.create_table(
    table,
    exists_ok=False,
    retry=<google.api_core.retry.Retry object at 0x7f0887682150>,
    timeout=None,
)
Docstring:
API call:  create a table via a PUT request

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/tables/insert

Args:
    table (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
        A :class:`~google.cloud.bigquery.table.Table` to create.
        If ``table`` is a reference, an empty table is created
        with the specified ID. The dataset that the table belongs to
        must already exist.
    exists_ok (Optional[bool]):
        Defaults to ``False``. If ``True``, ignore "already exists"
        errors when creating the table.
    retry (Optional[google.api_core.retry.Retry]):
        How to retry the RPC.
    timeout (Optional[float]):
        The number of seconds to wait for the underlying HTTP trans

In [16]:
abbr = 'RI'
file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
file.is_file()


True

In [14]:
def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            col = f.lower().split('_')[-1][:-4]
            print(col)
            if f[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(zipfile, f, sep=sep)
                
                if col == 'vtd':
                    display(df.head(2))
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        display(df.head(5))
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)
        display(assignments.head(2))

Get Rhode Island 2020 assignments.../home/jupyter/raw_data/assignments/2020/RI

fetching zip from https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST44_RI.zip...success
cdp
mcd
vtd


,blockid,countyfp,district
0,440010301001000,001,440101
1,440010301001001,001,440101


cd
sldu
sldl
sdsec
sdelm
sduni
aiannh


,geoid,cdp,mcd,vtd,cd,sldu,sldl,sdsec,sdelm,sduni
0,440010301001000,NaN,001,0010101,01,032,066,NaN,NaN,00030
1,440010301001001,NaN,001,0010101,01,032,066,NaN,NaN,00030
2,440010301001002,NaN,001,0010101,01,032,066,NaN,NaN,00030
3,440010301001003,NaN,001,0010101,01,032,066,NaN,NaN,00030
4,440010301001004,NaN,001,0010101,01,032,066,NaN,NaN,00030


,geoid,cdp,mcd,vtd,cd,sldu,sldl,sdsec,sdelm,sduni
0,440010301001000,NaN,001,0010101,01,032,066,NaN,NaN,00030
1,440010301001001,NaN,001,0010101,01,032,066,NaN,NaN,00030


...

Get Texas 2010 assignments.../home/jupyter/raw_data/assignments/2010/TX

fetching zip from https://www2.census.gov/geo/docs/maps-data/data/baf/BlockAssign_ST48_TX.zip...success
vtd


,blockid,countyfp,district
0,480019501001000,001,0018
1,480019501001001,001,0018


aiannh
cd
cdp
sdelm
sdsec
sduni
sldl
sldu


,geoid,vtd,cd,cdp,sdelm,sdsec,sduni,sldl,sldu
0,480019501001000,0010018,05,NaN,NaN,NaN,19770,008,003
1,480019501001001,0010018,05,27300,NaN,NaN,19770,008,003
2,480019501001002,0010018,05,27300,NaN,NaN,19770,008,003
3,480019501001003,0010018,05,27300,NaN,NaN,19770,008,003
4,480019501001004,0010018,05,NaN,NaN,NaN,19770,008,003


,geoid,vtd,cd,cdp,sdelm,sdsec,sduni,sldl,sldu
0,480019501001000,0010018,05,NaN,NaN,NaN,19770,008,003
1,480019501001001,0010018,05,27300,NaN,NaN,19770,008,003


In [15]:
def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            w = f.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(zipfile, f)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, parquetfile)
    return df
elections = get_elections_TX(overwrite)
display(elections.head(2))

Get Texas 2020 elections.../home/jupyter/raw_data/elections/2020/TX

fetching zip from https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/2020-general-vtd-election-data.zip...success


,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race
0,Anderson,1,0001,0010001,President,Ely,D,N,2,2012,Democratic_Primary_Election
1,Anderson,1,0002,0010002,President,Ely,D,N,5,2012,Democratic_Primary_Election
2,Anderson,1,0003,0010003,President,Ely,D,N,1,2012,Democratic_Primary_Election
3,Anderson,1,0004,0010004,President,Ely,D,N,2,2012,Democratic_Primary_Election
4,Anderson,1,0005,0010005,President,Ely,D,N,1,2012,Democratic_Primary_Election


,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race
0,Anderson,1,0001,0010001,President,Ely,D,N,2,2012,Democratic_Primary_Election
1,Anderson,1,0002,0010002,President,Ely,D,N,5,2012,Democratic_Primary_Election


In [ ]:
def get_shapefiles(state, yr, level, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    levels = ['tract', 'bg', 'tabblock']
    assert level in levels, f'level must be in {levels} - got {level}'
    
    name = f'shapefiles_{level}'
    url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{level.upper()}"
    if yr == 2010:
        url += '/2010'
    elif yr == 2020 and level == 'tabblock':
        url += '20'
    url += f"/tl_{yr}_{state['fips']}_{level}{str(yr)[-2:]}"
    if yr == 2020 and level in ['tract', 'bg']:
        url = url[:-2]
    url += '.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        path = parquetfile.parent
        zipfile.extractall(path)
        df = gpd.read_file(path)
        df.columns = [x[:-2].lower() if x[-2:].isnumeric() else x.lower() for x in df.columns]
        df = df[['geoid', 'aland', 'geometry']]
        df['geometry'] = df['geometry'].apply(lambda p: orient(p, -1))
        write_data(df.to_wkt(), parquetfile)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        for level in ['tract', 'bg']:#, 'tabblock']:
            geo = get_shapefiles(state, yr, level, overwrite)

# old

In [ ]:
def get_geo(state, yr, level):
    yr = str(yr)
    if level == 'blockgroup':
        a = 'bg'
    elif level == 'block':
        a = 'tabblock'
    else:
        a = level

    path = root_path / f"geo/{level}/{yr}/{state['abbr']}"
    path.mkdir(exist_ok=True, parents=True)
    try:
        geo = gpd.read_file(path)
    except:
        %cd {path}
        url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{a.upper()}"
        
        if yr == '2010':
            url += '/2010'
        elif yr == '2020' and level == 'block':
            url += '20'
        url += f"/tl_{yr}_{state['fips']}_{a}{yr[-2:]}"
        if yr == '2020' and level in ['blockgroup', 'tract']:
            url = url[:-2]
        url += '.zip'
        ! wget -O geo.zip {url}
        ! unzip geo.zip
        ! rm geo.zip
        geo = gpd.read_file(path)

    geo = lower_cols(geo)[['geoid', 'aland', 'geometry']].apply(lambda p: orient(p, -1))
    geo.columns = ['geo_id', 'geometry']

    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    job.result()
    return geo


for i, state in states.iterrows():
    print(state['name'])
    for level in ['tract', 'blockgroup', 'block']:
        print(level)
        for yr in [2010, 2020]:
            print(yr)
            geo = get_geo(state, yr, level)

In [ ]:
def read_geo(state, yr, level):
    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    qry = f"""
        select
            *
        from
            {table_id}
        """
#     job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    
#     job.result()  # Wait for the job to complete.
    bgs = bqclient.query(qry).result().to_dataframe()
    bgs['geometry'] = gpd.GeoSeries.from_wkt(bgs['geometry'])
    return bgs
#         table = bqclient.get_table(table_id)  # Make an API request.

state = states.iloc[0]
geo = get_geo(state, yr, level)
geo